In [73]:
import pandas as pd
import numpy as np
import random
#from catboost import CatBoostRanker, Pool, MetricVisualizer
#from copy import deepcopy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [80]:
HotelSearch_train=pd.read_csv("./data/hotelsearch_clf_training.csv", index_col=0)
HotelSearch_test=pd.read_csv("./data/hotelsearch_clf_test.csv", index_col=0)

# HotelSearch_train["PredictedRelevancy"]=round(HotelSearch_train["PredictedRelevancy"],4)
# HotelSearch_test["PredictedRelevancy"]=round(HotelSearch_test["PredictedRelevancy"],4)
HotelSearch_train.head()
HotelSearch_train["ActualRelevancy"].value_counts()

0.0    89823
1.0     2628
Name: ActualRelevancy, dtype: int64

In [81]:
def transpose(df):
    return pd.DataFrame(
        {"sequence": [df["ActualRelevancy"].tolist()] + [None] * (len(df) - 1)}, index=df.index
    )


HotelSearch_train = pd.concat([HotelSearch_train, HotelSearch_train.groupby("srch_id").apply(transpose)], axis=1)
HotelSearch_test = pd.concat([HotelSearch_test, HotelSearch_test.groupby("srch_id").apply(transpose)], axis=1)
HotelSearch_test.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,ActualRelevancy,Not_Relevant_prob,Yes_Relevant_prob,RelevancePrediction,sequence
0,246.0,14.0,100.0,NaN,NaN,219.0,11396.0,4.0,5.0,0.0,0.69,0.0551,5.36,1.0,159.64,0.0,18294.0,1.0,6.0,2.0,0.0,1.0,0.0,NaN,87.17,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.853760,0.146240,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,246.0,14.0,100.0,NaN,NaN,219.0,103885.0,2.0,4.0,1.0,1.10,0.1540,4.75,2.0,98.32,0.0,18294.0,1.0,6.0,2.0,0.0,1.0,0.0,NaN,93.87,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.852586,0.147414,0.0,None
2,246.0,14.0,100.0,NaN,NaN,219.0,61167.0,3.0,4.5,0.0,1.10,0.0314,4.87,3.0,101.24,0.0,18294.0,1.0,6.0,2.0,0.0,1.0,0.0,NaN,92.31,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.906396,0.093604,0.0,None
3,246.0,14.0,100.0,NaN,NaN,219.0,95490.0,2.0,3.5,1.0,1.10,0.0142,4.88,4.0,95.40,0.0,18294.0,1.0,6.0,2.0,0.0,1.0,0.0,NaN,89.28,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.925151,0.074849,0.0,None
4,246.0,14.0,100.0,NaN,NaN,219.0,127213.0,3.0,4.0,1.0,1.10,0.1652,4.95,6.0,119.73,0.0,18294.0,1.0,6.0,2.0,0.0,1.0,0.0,NaN,93.97,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.923575,0.076425,0.0,None


In [82]:
HotelSearch_train[["srch_id","site_id","position","ActualRelevancy","RelevancePrediction","sequence"]][HotelSearch_train.srch_id ==71]

,srch_id,site_id,position,ActualRelevancy,RelevancePrediction,sequence
0,71.0,24.0,1.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,71.0,24.0,2.0,0.0,0.0,None
2,71.0,24.0,3.0,0.0,0.0,None
3,71.0,24.0,4.0,0.0,0.0,None
4,71.0,24.0,6.0,0.0,0.0,None
5,71.0,24.0,7.0,0.0,0.0,None
6,71.0,24.0,8.0,0.0,0.0,None
7,71.0,24.0,9.0,0.0,0.0,None
8,71.0,24.0,10.0,0.0,0.0,None
9,71.0,24.0,12.0,0.0,0.0,None


In [85]:
HotelSearch_test[["srch_id","site_id","position","ActualRelevancy","RelevancePrediction","sequence"]][HotelSearch_test.srch_id ==246]

,srch_id,site_id,position,ActualRelevancy,RelevancePrediction,sequence
0,246.0,14.0,1.0,1.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,246.0,14.0,2.0,0.0,0.0,None
2,246.0,14.0,3.0,0.0,0.0,None
3,246.0,14.0,4.0,0.0,0.0,None
4,246.0,14.0,6.0,0.0,0.0,None
5,246.0,14.0,7.0,0.0,0.0,None
6,246.0,14.0,8.0,0.0,0.0,None
7,246.0,14.0,9.0,0.0,0.0,None
8,246.0,14.0,10.0,0.0,0.0,None
9,246.0,14.0,12.0,0.0,0.0,None
